In [1]:
%load_ext autoreload
%autoreload 2

# 1. Patch descriptors register in 3D for 642 templates

In [2]:
import glob
import os

# Load templates
template_path = "datasets/bop23_challenge/datasets/templates_pyrender/icbin_642/obj_000001"
template_files = sorted(glob.glob(os.path.join(template_path, "*.png")), key=os.path.getmtime)

In [3]:
from PIL import Image
import numpy as np

templates = [np.array(Image.open(template_file).convert("RGB").resize((420,420)))[:,:,:3] for template_file in template_files] # This image has 4 channels- the last one is not crucial - maybe about opacity

In [5]:
import torchvision.transforms as T
import torch.nn.functional as F
import torch

def patches_feature_extraction(template_patches, dinov2_vitl14):
    # crop_rgb: numpy array
    # temps = np.transpose(np.array(template_patches), (0,2,3,1))
    rgb_normalize = T.Compose(
        [
            T.ToTensor(),
            T.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
        ]
    )
    normalized_patches = torch.stack([rgb_normalize(patch) for patch in template_patches])
    layers_list = list(range(18))

    with torch.no_grad(): 
        feature_patches= dinov2_vitl14.get_intermediate_layers(normalized_patches.to("cuda:1"), n=layers_list, reshape=True)
    return feature_patches

dinov2_vitl14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14').to("cuda:1")
dinov2_vitl14.patch_size = 14


Using cache found in /home/cuong.vandam/.cache/torch/hub/facebookresearch_dinov2_main


In [1]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    dinov2_vitl14 = torch.nn.DataParallel(dinov2_vitl14).to(device)  # Use DataParallel for multiple GPUs

NameError: name 'torch' is not defined

In [8]:
batch_size = 4
template_batches = [templates[i:i+batch_size] for i in range(0, len(templates), batch_size)]
patch_features= list()

for batch in template_batches:
    torch.cuda.empty_cache()
    batch_feature = patches_feature_extraction(batch, dinov2_vitl14)
    patch_features.append(batch_feature)

OutOfMemoryError: CUDA out of memory. Tried to allocate 58.00 MiB (GPU 1; 23.59 GiB total capacity; 22.72 GiB already allocated; 20.31 MiB free; 23.18 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [9]:
batch_feature = patches_feature_extraction(templates[:100], dinov2_vitl14)

OutOfMemoryError: CUDA out of memory. Tried to allocate 202.00 MiB (GPU 1; 23.59 GiB total capacity; 22.72 GiB already allocated; 20.31 MiB free; 23.18 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
patch_features = torch.tensor(patch_features)

AttributeError: 'DataParallel' object has no attribute 'get_intermediate_layers'

In [ ]:
patches_features[0].shape

torch.Size([100, 1024, 30, 30])